# Сбор данных о членах Совета Республики НС VI созыва

Ссылка - http://sovrep.gov.by/ru/senators-ru/

In [37]:
from bs4 import BeautifulSoup as bs
import requests
import json
import re
from time import sleep

In [ ]:
def save_image(image_url, image_name):
    image = requests.get(image_url).content
    image_ext = re.match("^.+\.([a-z]+)$", image_url).group(1)
    with open("sovrep/img/{}.{}".format(image_name, image_ext), "wb") as f:
        f.write(image)

def get_aides(aides):
    try:
        return { aide.find("div", {"class": "assistant_title"}).get_text().strip(): aide.find("p").get_text().strip() for aide in aides }
    except AttributeError:
        return { aide.find("div", {"class": "assistant_title"}).get_text().strip(): aide.get_text().strip() for aide in aides }
    except TypeError:
        return None

def get_personal_data(link):
    page_id = re.match("^.+/(.+)/$", link).group(1)
    page = requests.get(link).content
    soup = bs(page, "html5lib")
    image_url = soup.find("div", {"class": "img_inner"}).find("img")["src"]
    image_ext = re.match("^.+\.([a-z]+)$", image_url).group(1)
    save_image(image_url, page_id)
    
    main_info = soup.find_all("div", {"class": "person_text"})
    position = soup.find_all("div", {"class": "person_text"})[0].get_text().strip()
    
    if len(main_info) > 1:
        try:
            commission = soup.find_all("div", {"class": "person_text"})[1].find("a").text.strip()
        except AttributeError:
            commission = None
    else:
        commission = None
    
    bio = soup.find("div", {"id": "biography_info"}).get_text().strip()
    
    return {
        "page_id": page_id,
        "image": "{}.{}".format(page_id, image_ext),
        "general_info": position,
        "commission": commission,
        "bio": bio,
        "aides": get_aides(aides)
    }

In [2]:
url_main = "http://sovrep.gov.by/ru/senators-ru/"
soup = bs(requests.get(url_main).content, "html5lib")
people = soup.find("div", {"class": "left_col"}).find_all("div", {"class": "news_title"})

In [89]:
data = {}
for person in people:
    fio = person.find("a").get_text().strip()
    url = person.find("a")["href"]
    if fio not in data:
        try:
            data[fio] = get_personal_data(url)
        except TypeError:
            print(url)
            continue
    sleep(2)

In [90]:
print(len(people), len(data.keys()))

56 56


In [92]:
with open("sovrep/VI sozyv/senatory.json", "w") as f:
    f.write(json.dumps(senatory_2019, ensure_ascii=False))